<img src='https://encrypted-tbn0.gstatic.com/images?q=tbn%3AANd9GcQ-VfNtOyJbsaxu43Kztf_cv1mgBG6ZIQZEVw&usqp=CAU'>

# Procesamiento de Lenguage Natural

## Taller #8: Modelado de temas

### Jairo Iván Ordóñez
### Grupo 51

`Fecha de entrega: 🎃Octubre 31, 2020. (Antes del inicio de la próxima clase).`

`Modo de entrega: Subir link de GitHub al aula virtual.`

In [1]:
import re
import pandas as pd 
from pprint import pprint

from nltk.corpus import stopwords
stopwords = stopwords.words('spanish')

import pyLDAvis.gensim
from gensim.models import LdaModel
from gensim.corpora import Dictionary

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

C:\Jairo.Ordonez\2. Personal\Data Science Program\Anaconda3\lib\site-packages\scipy\sparse\sparsetools.py:21: DeprecationWarning: `scipy.sparse.sparsetools` is deprecated!
scipy.sparse.sparsetools is a private module for scipy.sparse, and should not be used.
  _deprecated()


In [2]:
# Cargar datos
path = './reviews_vidjew_es.csv'
data = pd.read_csv(path)
data.head()

,review_id,product_id,reviewer_id,stars,review_body,review_title,language,product_category
0,es_0825565,product_es_0370490,reviewer_es_0174781,3,"Buen. Buena calidad, y buena presentación.",Contenta,es,jewelry
1,es_0227934,product_es_0354224,reviewer_es_0411613,3,"Un producto a perfecto, para salir de casa con...",Versatilidad,es,video_games
2,es_0468601,product_es_0665460,reviewer_es_0348315,1,No funciona con Nintendo Switch. No hay forma ...,Decepción absoluta,es,video_games
3,es_0814494,product_es_0692692,reviewer_es_0951508,5,"Recomendado, los utilizo para pc y no me dan n...",Auriculares Pecham ps4,es,video_games
4,es_0206329,product_es_0728826,reviewer_es_0493255,4,El cable funciona bien podria ser un poco mas ...,Perfecto,es,video_games


###  `[12 pts]` Punto 1: Hacer pre-procesamiento del texto

In [3]:
def pre_procesado(texto):
    texto = texto.lower()
    texto = re.sub(r"[\W\d_]+", " ", texto)
    texto = [palabra for palabra in texto.split() if palabra not in stopwords]
    return texto

data['pp'] = data['review_body'].apply(lambda texto: pre_procesado(texto))

In [4]:
data.head()

,review_id,product_id,reviewer_id,stars,review_body,review_title,language,product_category,pp
0,es_0825565,product_es_0370490,reviewer_es_0174781,3,"Buen. Buena calidad, y buena presentación.",Contenta,es,jewelry,"[buen, buena, calidad, buena, presentación]"
1,es_0227934,product_es_0354224,reviewer_es_0411613,3,"Un producto a perfecto, para salir de casa con...",Versatilidad,es,video_games,"[producto, perfecto, salir, casa, nintendo, sw..."
2,es_0468601,product_es_0665460,reviewer_es_0348315,1,No funciona con Nintendo Switch. No hay forma ...,Decepción absoluta,es,video_games,"[funciona, nintendo, switch, forma, emparejarl..."
3,es_0814494,product_es_0692692,reviewer_es_0951508,5,"Recomendado, los utilizo para pc y no me dan n...",Auriculares Pecham ps4,es,video_games,"[recomendado, utilizo, pc, dan, ningún, proble..."
4,es_0206329,product_es_0728826,reviewer_es_0493255,4,El cable funciona bien podria ser un poco mas ...,Perfecto,es,video_games,"[cable, funciona, bien, podria, ser, mas, larg..."


###  `[13 pts]` Punto 2: Modelo de LDA

In [5]:
# Crear una representación de los documentos en forma de diccionario
diccionario = Dictionary(data['pp'].values)

# Filtrar palabras muy frecuentes o infrecuentes
diccionario.filter_extremes(no_below= 4, no_above=0.7)

corpus = [diccionario.doc2bow(text) for text in data['pp'].values]

#Train the topic model
model = LdaModel(corpus=corpus, id2word=diccionario, num_topics=3, passes=100)

In [6]:
model.print_topics(num_words=10)

[(0,
  '0.030*"bien" + 0.026*"calidad" + 0.022*"si" + 0.019*"precio" + 0.017*"mando" + 0.017*"foto" + 0.014*"mas" + 0.011*"queda" + 0.011*"buena" + 0.011*"bonitos"'),
 (1,
  '0.037*"bien" + 0.022*"calidad" + 0.019*"regalo" + 0.018*"llegado" + 0.014*"plata" + 0.013*"bonito" + 0.013*"producto" + 0.012*"bastante" + 0.012*"dos" + 0.011*"pendientes"'),
 (2,
  '0.040*"juego" + 0.021*"perfecto" + 0.020*"producto" + 0.017*"precio" + 0.016*"caja" + 0.013*"si" + 0.013*"tiempo" + 0.012*"día" + 0.010*"llegó" + 0.009*"bonita"')]

###  `[25 pts]` Punto 3: Visualización de LDA

In [7]:
lda_display = pyLDAvis.gensim.prepare(model, corpus, diccionario, sort_topics=False)
pyLDAvis.display(lda_display)